In [1]:
import urllib
import zipfile
import geopandas as gpd
import pandas as pd

/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
url_download = r'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/ageb_manzana/RESAGEBURB_16_2020_csv.zip'
input_path = 'inputs/'
urllib.request.urlretrieve(url_download, 'RESAGEBURB_16_2020_csv.zip')
unzip = zipfile.ZipFile('RESAGEBURB_16_2020_csv.zip')
unzip.extractall(input_path)

In [3]:
url_download = r'https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463849568/16_michoacandeocampo.zip'
input_path = 'inputs/'
urllib.request.urlretrieve(url_download, '16_michoacandeocampo.zip')
unzip = zipfile.ZipFile('16_michoacandeocampo.zip')
unzip.extractall(input_path)

In [4]:
michoacan_full = pd.read_csv('inputs/RESAGEBURB_16CSV20.csv')

In [5]:
michoacan_shp = gpd.read_file('inputs/conjunto_de_datos/16m.shp')

In [6]:
morelia = michoacan_full.loc[(michoacan_full["MUN"] ==53) & (michoacan_full["MZA"] !=0) ]

In [7]:
morelia.to_csv('outputs/morelia_manzanas_original.csv')

In [8]:
morelia["ENTIDAD"] = morelia["ENTIDAD"].astype(str)

/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
morelia['MUN'] = morelia['MUN'].apply(lambda x: '{0:0>3}'.format(x))
morelia['LOC'] = morelia['LOC'].apply(lambda x: '{0:0>4}'.format(x))
morelia['MZA'] = morelia['MZA'].apply(lambda x: '{0:0>3}'.format(x))
morelia['CVEGEO']= morelia['ENTIDAD'] + morelia['MUN'] +morelia['LOC'] +morelia['AGEB'] +morelia['MZA']


/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [10]:
morelia_shp = michoacan_shp.loc[(michoacan_shp["CVE_MUN"] =='053') ]

In [11]:
morelia_shp.to_file('outputs/morelia_original.shp')

In [12]:
shape_full = pd.merge(morelia_shp, morelia[['MUN','LOC','MZA','CVEGEO','POBTOT','P_18YMAS','TVIVHAB']], on = 'CVEGEO')

In [13]:
shape_full.to_file('outputs/morelia_con_poblacion.shp')

In [14]:
morelia_urbano = shape_full.loc[(shape_full["AMBITO"] =='Urbana') ]


In [15]:
morelia_urbano = morelia_urbano.replace({'*': '0'})

In [16]:
morelia_urbano_habitado = morelia_urbano.loc[(morelia_urbano["TVIVHAB"] !=0) ]

In [18]:
morelia_urbano_habitado.to_file('outputs/morelia_urbano_habitado.shp')
morelia_urbano_habitado.to_csv('outputs/morelia_urbano_habitado.csv')

In [20]:
colonias_michoacan = gpd.read_file("inputs/COLONIA.shp" )

In [21]:
colonias_morelia = colonias_michoacan.loc[(colonias_michoacan["MUNICIPIO"] ==54) ]


In [22]:
colonias_morelia = colonias_morelia.to_crs('EPSG:4326')


In [23]:
morelia_urbano_habitado = morelia_urbano_habitado.to_crs('EPSG:4326')

In [24]:
morelia_urbano_habitado['centroid'] = morelia_urbano_habitado.centroid
join = gpd.sjoin(colonias_morelia, morelia_urbano_habitado.set_geometry('centroid'), how="right", predicate="contains")


/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [25]:
join.P_18YMAS = join.P_18YMAS.fillna(0)

In [26]:
join.TVIVHAB = join.TVIVHAB.fillna(0)

In [27]:
join.P_18YMAS = join.P_18YMAS.astype('int64')

In [28]:
join.TVIVHAB = join.TVIVHAB.astype('int64')

In [29]:
habitantes_colonia = join.groupby(['NOMBRE'])[['POBTOT', "P_18YMAS", "TVIVHAB"]].agg('sum').reset_index()

In [30]:
pob_col_mor = pd.merge(colonias_morelia, habitantes_colonia,on='NOMBRE')

In [31]:
pob_col_mor.to_file('outputs/poblacion_colonias_morelia.shp')
pob_col_mor.to_csv('outputs/poblacion_colonias_morelia.csv')

In [32]:

cuadrantes_morelia = gpd.read_file("inputs/redudasproyectomorelia (1)/sectores_processed.shp" )




In [33]:
cuadrantes_morelia = cuadrantes_morelia.to_crs('EPSG:4326')

In [34]:
pob_col_mor['centroid'] = pob_col_mor.centroid

/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [35]:

join_cuadrantes = gpd.sjoin(cuadrantes_morelia, pob_col_mor.set_geometry('centroid'), how="left", predicate="contains")

In [39]:

join_cuadrantes.to_csv('outputs/colonias_con_poblacion_y_cuadrantes.csv')

In [40]:

tabla_cuadrantes = join_cuadrantes.groupby('Sector')['POBTOT',"P_18YMAS",'TVIVHAB'].sum()

/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [41]:
tabla_cuadrantes.to_csv('outputs/tabla_poblacion.csv')